<center>
    This is Set 5: Column Aggregations (Exercises 41-50)
</center>

**Prerequisites**
* The sample dataset [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) from sklearn will be used for the exercises.

In [3]:
# !pip3 install scikit-learn==1.0.2


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/27 16:48:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import pandas as pd
from sklearn.datasets import load_wine

data = spark.createDataFrame(load_wine(as_frame=True).frame)
data.show()


+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+----------------------------+-------+------+
|alcohol|malic_acid| ash|alcalinity_of_ash|magnesium|total_phenols|flavanoids|nonflavanoid_phenols|proanthocyanins|color_intensity| hue|od280/od315_of_diluted_wines|proline|target|
+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+----------------------------+-------+------+
|  14.23|      1.71|2.43|             15.6|    127.0|          2.8|      3.06|                0.28|           2.29|           5.64|1.04|                        3.92| 1065.0|     0|
|   13.2|      1.78|2.14|             11.2|    100.0|         2.65|      2.76|                0.26|           1.28|           4.38|1.05|                         3.4| 1050.0|     0|
|  13.16|      2.36|2.67|             18.6|    101.0|          2.8|      3.24|                 

**Exercise 41: Calculate the sum of column `ash` in `data`**

In [3]:
data.agg(F.sum(data["ash"])).collect()[0][0]


421.24

**Exercise 42: Calculate the average value of each column in `data`**

In [4]:
data.agg(*[F.mean(col).alias(col) for col in data.columns]).first().asDict()


22/07/27 16:48:24 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


{'alcohol': 13.00061797752809,
 'malic_acid': 2.3363483146067416,
 'ash': 2.3665168539325845,
 'alcalinity_of_ash': 19.49494382022472,
 'magnesium': 99.74157303370787,
 'total_phenols': 2.295112359550562,
 'flavanoids': 2.0292696629213487,
 'nonflavanoid_phenols': 0.36185393258426973,
 'proanthocyanins': 1.590898876404494,
 'color_intensity': 5.058089882022472,
 'hue': 0.9574494382022471,
 'od280/od315_of_diluted_wines': 2.611685393258427,
 'proline': 746.8932584269663,
 'target': 0.9382022471910112}

**Exercise 43: Calculate the minimum value and maximum value of each column in `data` where `target` is 1**

In [5]:
data.where(data["target"] == 1).groupBy().min().first().asDict()


{'min(alcohol)': 11.03,
 'min(malic_acid)': 0.74,
 'min(ash)': 1.36,
 'min(alcalinity_of_ash)': 10.6,
 'min(magnesium)': 70.0,
 'min(total_phenols)': 1.1,
 'min(flavanoids)': 0.57,
 'min(nonflavanoid_phenols)': 0.13,
 'min(proanthocyanins)': 0.41,
 'min(color_intensity)': 1.28,
 'min(hue)': 0.69,
 'min(od280/od315_of_diluted_wines)': 1.59,
 'min(proline)': 278.0,
 'min(target)': 1}

In [6]:
data.where(data["target"] == 1).groupBy().max().first().asDict()


{'max(alcohol)': 13.86,
 'max(malic_acid)': 5.8,
 'max(ash)': 3.23,
 'max(alcalinity_of_ash)': 30.0,
 'max(magnesium)': 162.0,
 'max(total_phenols)': 3.52,
 'max(flavanoids)': 5.08,
 'max(nonflavanoid_phenols)': 0.66,
 'max(proanthocyanins)': 3.58,
 'max(color_intensity)': 6.0,
 'max(hue)': 1.71,
 'max(od280/od315_of_diluted_wines)': 3.69,
 'max(proline)': 985.0,
 'max(target)': 1}

**Exercise 44: Calculate the median value of columns `flavanoids`, `hue` and `magnesium` in `data`**

In [7]:
cols = ["flavanoids", "hue", "magnesium"]

data.approxQuantile(cols, [0.5], 0.0)


[[2.13], [0.96], [98.0]]

**Exercise 45: Calculate the mode of columns `alcohol` and `proline` in `data`**

In [8]:
[
    [i, data.groupby(i).count().orderBy("count", ascending=False).first()[0]]
    for i in ["alcohol", "proline"]
]


[['alcohol', 13.05], ['proline', 680.0]]

**Exercise 46: Calculate the standard deviation, skewness and excess kurtosis of each column in `data`**

In [15]:
std = data.agg(*[F.stddev(col).alias(col) for col in data.columns]).first().asDict()
skew = data.agg(*[F.skewness(col).alias(col) for col in data.columns]).first().asDict()
kurtosis = (
    data.agg(*[F.kurtosis(col).alias(col) for col in data.columns]).first().asDict()
)

std, skew, kurtosis


({'alcohol': 0.8118265380058572,
  'malic_acid': 1.1171460976144627,
  'ash': 0.2743440090608149,
  'alcalinity_of_ash': 3.3395637671735043,
  'magnesium': 14.282483515295668,
  'total_phenols': 0.625851048833989,
  'flavanoids': 0.9988586850169469,
  'nonflavanoid_phenols': 0.12445334029667937,
  'proanthocyanins': 0.5723588626747612,
  'color_intensity': 2.3182858718224124,
  'hue': 0.22857156582982333,
  'od280/od315_of_diluted_wines': 0.7099904287650505,
  'proline': 314.9074742768491,
  'target': 0.7750349899850563},
 {'alcohol': -0.05104747165391558,
  'malic_acid': 1.0308694978039965,
  'ash': -0.1752067779255995,
  'alcalinity_of_ash': 0.21124732832155288,
  'magnesium': 1.0889148872107,
  'total_phenols': 0.08590677016719506,
  'flavanoids': 0.025129482208429043,
  'nonflavanoid_phenols': 0.44634901064702137,
  'proanthocyanins': 0.5127690334109941,
  'color_intensity': 0.8612480547740538,
  'hue': 0.020913119068882262,
  'od280/od315_of_diluted_wines': -0.3046899289436109,
  

**Exercise 47: Calculate the number of unique values in each column of `data`**

In [18]:
data.agg(*(F.countDistinct(F.col(c)).alias(c) for c in data.columns)).show()


+-------+----------+---+-----------------+---------+-------------+----------+--------------------+---------------+---------------+---+----------------------------+-------+------+
|alcohol|malic_acid|ash|alcalinity_of_ash|magnesium|total_phenols|flavanoids|nonflavanoid_phenols|proanthocyanins|color_intensity|hue|od280/od315_of_diluted_wines|proline|target|
+-------+----------+---+-----------------+---------+-------------+----------+--------------------+---------------+---------------+---+----------------------------+-------+------+
|    126|       133| 79|               63|       53|           97|       132|                  39|            101|            132| 78|                         122|    121|     3|
+-------+----------+---+-----------------+---------+-------------+----------+--------------------+---------------+---------------+---+----------------------------+-------+------+



**Exercise 48: Calculate the covariance between columns `alcohol` and `color_intensity` in `data`**

In [21]:
data.agg(F.covar_samp("alcohol", "color_intensity").alias("cov")).show()


+------------------+
|               cov|
+------------------+
|1.0282825443106072|
+------------------+



**Exercise 49: Calculate the correlation between columns `hue` and `color_intensity` in `data`**

In [22]:
data.agg(F.corr("hue", "color_intensity").alias("corr")).show()


+-------------------+
|               corr|
+-------------------+
|-0.5218131932287573|
+-------------------+



**Exercise 50: Calculate the correlation matrix between all pairs of columns in `data` and assign it to `corr_matrix`**

In [24]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

# convert to vector column first
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=data.columns, outputCol=vector_col)
df_vector = assembler.transform(data).select(vector_col)

# get correlation matrix
matrix = Correlation.corr(df_vector, vector_col)


In [32]:
matrix.collect()[0][0].values


array([ 1.        ,  0.09439694,  0.2115446 , -0.31023514,  0.27079823,
        0.28910112,  0.23681493, -0.15592947,  0.13669791,  0.5463642 ,
       -0.0717472 ,  0.07234319,  0.64372004, -0.32822194,  0.09439694,
        1.        ,  0.16404547,  0.2885004 , -0.0545751 , -0.335167  ,
       -0.41100659,  0.29297713, -0.22074619,  0.24898534, -0.56129569,
       -0.36871043, -0.19201056,  0.4377762 ,  0.2115446 ,  0.16404547,
        1.        ,  0.44336719,  0.28658669,  0.12897954,  0.11507728,
        0.18623045,  0.00965194,  0.25888726, -0.07466689,  0.00391123,
        0.22362626, -0.04964322, -0.31023514,  0.2885004 ,  0.44336719,
        1.        , -0.08333309, -0.32111332, -0.35136986,  0.36192172,
       -0.19732684,  0.01873198, -0.27395522, -0.27676855, -0.44059693,
        0.51785911,  0.27079823, -0.0545751 ,  0.28658669, -0.08333309,
        1.        ,  0.21440123,  0.19578377, -0.25629405,  0.23644061,
        0.19995001,  0.0553982 ,  0.06600394,  0.39335085, -0.20

✅ This completes Set 5: Column Aggregations (Exercises 41-50)  

Original exercises for Datatable package can be found [here](https://github.com/vopani/datatableton).

Pandas version can be found [there](https://github.com/Extremesarova/pandas_exercises).